# Test data reading with plot_vtk_matplotlib

In [20]:
%matplotlib inline
import dolfin as df
import mshr
import numpy as np
import plot_vtk_matplotlib as pvm
from vtk.util.numpy_support import vtk_to_numpy
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from scipy.interpolate import griddata

In [50]:
import os
files = os.listdir(".")
for f in files[:15]:
    print(f)

.ipynb_checkpoints
cavity_0.vtk
cavity_100.vtk
cavity_150.vtk
cavity_200.vtk
cavity_250.vtk
cavity_300.vtk
cavity_350.vtk
cavity_400.vtk
cavity_450.vtk
cavity_50.vtk
cavity_500.vtk
fixedWalls
frontAndBack
movingWall


In [43]:
def getData_pvm(filename):
    vf_plot = pvm.plot_vtk_matplotlib(filename,z_max=1, z_min=-1, vtkfiletype="UnstructuredGrid"  )
    # Extract the data from the file
    vf_plot.extract_data()
    print "PVM:"
    print vf_plot.reader.GetOutput().GetPoints()
    nodes_vtk = vf_plot.reader.GetOutput().GetPoints().GetData()
    data_arrays = vf_plot.reader.GetOutput().GetPointData()
    pressure_vtk_array = data_arrays.GetArray(0) 
    speed_vtk_array = data_arrays.GetArray(1)
    nodes = vtk_to_numpy(nodes_vtk)
    p = vtk_to_numpy(pressure_vtk_array)
    U  = vtk_to_numpy(speed_vtk_array)
    return nodes, p, U

In [44]:
def getData_vtk(filename):
    # load a vtk file as input
    reader = vtk.vtkUnstructuredGridReader()
    reader.SetFileName(filename)
    reader.Update()
    print "VTK:"
    print reader.GetOutput().GetPoints()
    # Get the coordinates of nodes in the mesh
    nodes_vtk_array= reader.GetOutput().GetPoints().GetData()  
    pressure_vtk_array = reader.GetOutput().GetPointData().GetArray(0)
    speed_vtk_array = reader.GetOutput().GetPointData().GetArray(1)
    nodes = vtk_to_numpy(nodes_vtk_array)
    p = vtk_to_numpy(pressure_vtk_array)
    U  = vtk_to_numpy(speed_vtk_array)
    return nodes, p, U

In [46]:
def plot3D(nodes, a, vmin, vmax):
    cmap = mpl.cm.seismic
    color_map = plt.cm.get_cmap('plasma')
    fig = plt.figure(figsize=(14,10))
    ax = fig.add_subplot(111, projection='3d')
    #for i in range(0,len(nodes),2):
    splt = ax.scatter(nodes[:,0],nodes[:,1],nodes[:,2],s=60,
                   c = a,
                   cmap = color_map,
                   vmin = vmin,
                   vmax = vmax,
                   marker='o')
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    fig.colorbar(splt, shrink=0.5)
    plt.show()

In [47]:
def extractFlatData(nodes, p, U):
    merged = np.empty([len(nodes),6])
    merged[:,0] = p
    merged[:,1:3] = U[:,:2]
    merged[:,3:] = nodes
    # Merged z0 and z1 arrays should be same because of 
    # the model symmetry along z-axis
    merged_z0 = merged[np.where(merged[:,5] == 0)]
    merged_z1 = merged[np.where(merged[:,5] != 0)]
    # Extract nodes, p and U arrays from z0 array
    p = merged_z0[:,0]
    U = merged_z0[:,1:3]
    nodes = merged_z0[:,3:5]
    return nodes, p, U

def plot2D(nodes, a, vmin, vmax):     
    # 2d plotting
    fig = plt.figure(figsize=(8,6))
    cmap = mpl.cm.seismic
    color_map = plt.cm.get_cmap('plasma')
    axes = plt.gca()
    #axes.set_xlim([-.1,2.1])
    #axes.set_ylim([-.01,0.11])
    #plt.axis('off')
    sc = plt.scatter(nodes[:,0],nodes[:,1],
                     s=320,
#                     color=cmap(a / a_max),
                     c = a,
                     cmap = color_map,
                     vmin = vmin,
                     vmax = vmax,
                     linewidth=0, 
                     marker="s")
    plt.colorbar(sc)
    fig.tight_layout()

In [49]:
filename="cavity_50.vtk"
nodes, p, U = getData_pvm(filename)
nodes_v, p_v, U_v = getData_vtk(filename)
print(np.amin(p),np.amax(p))
print nodes
print(np.amin(p_v),np.amax(p_v))
#vmin = -4e-06
#vmax = 6e-06
#plot3D(nodes,p, vmin, vmax)
#nodes_half, p_half, U_half = extractFlatData(nodes, p, U)
#plot2D(nodes_half,p_half, vmin, vmax)

PVM:
vtkPoints (0x70c08e0)
  Debug: Off
  Modified Time: 3919
  Reference Count: 2
  Registered Events: (none)
  Data: 0x7c34740
  Data Array Name: Points
  Number Of Points: 882
  Bounds: 
    Xmin,Xmax: (-1.01414e+27, 7.71994e+35)
    Ymin,Ymax: (-1.01414e+27, 7.71994e+35)
    Zmin,Zmax: (0, 2.0717e-32)


VTK:
vtkPoints (0x6e5fbe0)
  Debug: Off
  Modified Time: 4133
  Reference Count: 2
  Registered Events: (none)
  Data: 0x7ae0240
  Data Array Name: Points
  Number Of Points: 882
  Bounds: 
    Xmin,Xmax: (-1.01414e+27, 7.71994e+35)
    Ymin,Ymax: (-1.01414e+27, 7.71994e+35)
    Zmin,Zmax: (0, 2.0717e-32)


(nan, nan)
[[  0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  2.07651474e-32   0.00000000e+00   0.00000000e+00]
 [  2.07170006e-32   0.00000000e+00   0.00000000e+00]
 ..., 
 [ -1.01414215e+27  -4.29492128e+08   2.07170006e-32]
 [  3.23715911e+17  -4.29492128e+08   2.07170006e-32]
 [ -4.29492128e+08  -4.29492128e+08   2.07170006e-32]]
(nan, nan)


In [3]:
filename="cavity_100.vtk"
# Load 
vf_plot = pvm.plot_vtk_matplotlib(filename,z_max=0.001, z_min=-0.001, vtkfiletype="UnstructuredGrid"  )
# Extract the data from the file
vf_plot.extract_data()

In [4]:
import vtk
reader = vtk.vtkUnstructuredGridReader()
reader.SetFileName(filename)
reader.Update()

In [5]:
nodes_array = reader.GetOutput().GetPoints().GetData()
data_arrays = reader.GetOutput().GetPointData()
print data_arrays
pressure_vtk_array = data_arrays.GetArray(0) 
speed_vtk_array = data_arrays.GetArray(1)

vtkPointData (0x6b54d40)
  Debug: Off
  Modified Time: 347
  Reference Count: 2
  Registered Events: (none)
  Number Of Arrays: 2
  Array 0 name = p
  Array 1 name = U
  Number Of Components: 4
  Number Of Tuples: 882
  Copy Tuple Flags: ( 1 1 1 1 1 0 1 1 )
  Interpolate Flags: ( 1 1 1 1 1 0 0 1 )
  Pass Through Flags: ( 1 1 1 1 1 1 1 1 )
  Scalars: (none)
  Vectors: (none)
  Normals: (none)
  TCoords: (none)
  Tensors: (none)
  GlobalIds: (none)
  PedigreeIds: (none)
  EdgeFlag: (none)


